In [1]:
import pandas as pd
import icd10
import numpy as np

import warnings
from causalnex.structure import StructureModel
warnings.filterwarnings("ignore")  # silence warnings

from IPython.display import Image
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

from mlxtend.preprocessing import TransactionEncoder

import numpy as np
import pandas as pd
from scipy.stats import norm
from zepid import RiskRatio


# Data Processing

In [2]:
data = pd.read_table('entire_data_participant.tsv')
data = data.rename(columns={"p41270": "Diagnoses - ICD10", "p31": "Sex", 'p34': 'Year of birth'})

In [3]:
data['icd'] = data ['Diagnoses - ICD10'].str.split('|')
data['age'] = 2023 - data['Year of birth']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['gender'] = le.fit_transform(data['Sex'])

In [4]:
data_icd = data[data['icd'].notnull()]

In [5]:
data_no = data[data['icd'].isnull()]

In [6]:
fp_pn =  list(data_icd['icd'])

te = TransactionEncoder()
te_ary = te.fit(fp_pn).transform(fp_pn)
df = pd.DataFrame(te_ary, columns=te.columns_)
df = df.astype(int)

In [7]:
d = pd.DataFrame(0, index=np.arange(len(data_no)), columns=df.columns)
df = pd.concat([df,d])

In [8]:
n = df.describe().T
#top = n.sort_values(by = 'mean', ascending = False)[:500]
top = n[n['mean'] >= 0.005]
test_top = df[top.index]

In [10]:
test_top = df[top.index]

In [11]:
test_top = test_top.reset_index()

In [12]:
data = pd.concat([data_icd,data_no]).reset_index()

In [13]:
df = pd.concat([test_top.iloc[:,1:], data[['age','gender']]], axis=1)

In [14]:
df['age'] = np.where(df['age'] > 65, 1, 0)

In [15]:
df = df.dropna(how='any',axis=0) 

In [110]:
#gold = ['F03','I10','E78.0','N17.9','J22','F41.9','I25.9','N18.3','E66.9','K59','J22','N18.9','N18.3',\
#        'I67.8','G20','I67.9']

# NOTEARS_alpha

In [21]:
import numpy as np
import scipy.linalg as slin
import scipy.optimize as sopt
from scipy.special import expit as sigmoid

import time


def notears_linear(X, lambda1, loss_type, max_iter=100, h_tol=1e-8, rho_max=1e+16, w_threshold=0.0):
    """Solve min_W L(W; X) + lambda1 ‖W‖_1 s.t. h(W) = 0 using augmented Lagrangian.

    Args:
        X (np.ndarray): [n, d] sample matrix
        lambda1 (float): l1 penalty parameter
        loss_type (str): l2, logistic, poisson
        max_iter (int): max num of dual ascent steps
        h_tol (float): exit if |h(w_est)| <= htol
        rho_max (float): exit if rho >= rho_max
        w_threshold (float): drop edge if |weight| < threshold

    Returns:
        W_est (np.ndarray): [d, d] estimated DAG
    """
    def _loss(W):
        """Evaluate value and gradient of loss."""
        M = X @ W
        if loss_type == 'l2':
            alpha = 0
            Y = 1 - X
            R = X *(X - M) + alpha * Y * (X - M)
            #R = X - M
            loss = 0.5 / X.shape[0] * (R ** 2).sum()
            G_loss = - 1.0 / X.shape[0] * X.T @ R
        elif loss_type == 'logistic':
            loss = 1.0 / X.shape[0] * (np.logaddexp(0, M) - X * M).sum()
            G_loss = 1.0 / X.shape[0] * X.T @ (sigmoid(M) - X)
        elif loss_type == 'poisson':
            S = np.exp(M)
            loss = 1.0 / X.shape[0] * (S - X * M).sum()
            G_loss = 1.0 / X.shape[0] * X.T @ (S - X)
        else:
            raise ValueError('unknown loss type')
        return loss, G_loss

    def _h(W):
        """Evaluate value and gradient of acyclicity constraint."""
        E = slin.expm(W * W)  # (Zheng et al. 2018)
        h = np.trace(E) - d
        #     # A different formulation, slightly faster at the cost of numerical stability
        #     M = np.eye(d) + W * W / d  # (Yu et al. 2019)
        #     E = np.linalg.matrix_power(M, d - 1)
        #     h = (E.T * M).sum() - d
        G_h = E.T * W * 2
        return h, G_h

    def _adj(w):
        """Convert doubled variables ([2 d^2] array) back to original variables ([d, d] matrix)."""
        return (w[:d * d] - w[d * d:]).reshape([d, d])

    def _func(w):
        """Evaluate value and gradient of augmented Lagrangian for doubled variables ([2 d^2] array)."""
        W = _adj(w)
        loss, G_loss = _loss(W)
        h, G_h = _h(W)
        obj = loss + 0.5 * rho * h * h + alpha * h + lambda1 * w.sum()
        G_smooth = G_loss + (rho * h + alpha) * G_h
        g_obj = np.concatenate((G_smooth + lambda1, - G_smooth + lambda1), axis=None)
        return obj, g_obj

    n, d = X.shape
    w_est, rho, alpha, h = np.zeros(2 * d * d), 1.0, 0.0, np.inf  # double w_est into (w_pos, w_neg)
    bnds = [(0, 0) if i == j else (0, None) for _ in range(2) for i in range(d) for j in range(d)]
    if loss_type == 'l2':
        X = X - np.mean(X, axis=0, keepdims=True)
    for iter in range(max_iter):
        print(iter)
        w_new, h_new = None, None
        while rho < rho_max:
            sol = sopt.minimize(_func, w_est, method='L-BFGS-B', jac=True, bounds=bnds,\
                                options={'disp': True,'eps':1e-40,'maxls':40})
            #sol = sopt.minimize(_func, w_est, method='BFGS', jac=True, bounds=bnds)
            #sol = sopt.fmin_l_bfgs_b(_func, w_est,approx_grad=True, bounds=[(1e-8, 0.5)], factr=1e02, pgtol=1e-05, epsilon=1e-08)

            w_new = sol.x
            h_new, _ = _h(_adj(w_new))
            if h_new > 0.25 * h:
                rho *= 10
            else:
                break
        w_est, h = w_new, h_new
        W_est = _adj(w_est)
        print(np.sum(W_est))
        alpha += rho * h
        if h <= h_tol or rho >= rho_max:
            break
    W_est = _adj(w_est)
    return W_est

def read_file(data_dir, sep=","):
    with open(data_dir, "r") as f:
        data = f.read().strip().split("\n")
    data = [i.split(sep) for i in data]
    return np.array(data).astype(float)

if __name__ == '__main__':
    # from tools import utils
    #
    # utils.set_random_seed(1)
    #
    # n, d, s0, graph_type, sem_type = 20000, 100, 20, 'SF', 'gauss' # sem_type: gauss, logistic
    # B_true = utils.simulate_dag(d, s0, graph_type)
    # print(B_true.sum())
    # W_true = B_true
    # #W_true = utils.simulate_parameter(B_true) # why should we simulate parameters?
    # #np.savetxt('W_true.csv', W_true, delimiter=',')
    #
    # X = utils.simulate_linear_sem(W_true, n, sem_type)
    #
    # X2 = utils.simulate_linear_sem(W_true, n, sem_type)
    # #X = utils.simulate_nonlinear_sem(W_true, n, sem_type)
    # #np.savetxt('X.csv', X, delimiter=',')
    #
    # start_time = time.time()
    #
    # W_est = notears_linear(X, lambda1=0., loss_type='l2', max_iter=10)
    #
    # end_time = time.time()
    # print("time: {}".format(end_time-start_time))
    #
    # print("W_est.sum: {}".format(W_est.sum()))
    # W_est[np.abs(W_est) < 0.1] = 0
    # assert utils.is_dag(W_est)
    # #np.savetxt('W_est.csv', W_est, delimiter=',')
    # acc = utils.count_accuracy(B_true, W_est != 0)
    # print(W_est.sum())
    # print(acc)
    ###############################################################

    X2 = df.to_numpy()
    W_est = notears_linear(X2, lambda1=0., loss_type='l2', max_iter=5)
    print("W_est: {}".format(W_est))

    import pandas as pd
    W_out = pd.DataFrame(W_est)
    W_out.to_csv('entire_biobank_10_encode_age_max_2.csv')
    # X = read_file("X_mimic.txt", " ")
    # W_est = notears_linear(X, lambda1=0., loss_type='l2', max_iter=100, w_threshold=0.)
    print(W_est)

    ##############################
    # import pandas as pd
    # mimic3 = read_file("DIAGNOSES_ICD.csv")
    # mimic3 = pd.DataFrame(mimic3[1:], columns=["ROW_ID", "SUBJECT_ID", "HADM_ID", "SEQ_NUM", "ICD9_CODE"])
    # patients = mimic3['SUBJECT_ID'].to_list()
    # patients = list(set(patients))
    # print(len(patients))
    # icd = read_file("icd.csv")
    # icd = np.array(icd).astype(int)

0
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       462722     M =           10

At X0    462722 variables are exactly at the bounds

At iterate    0    f=  4.57901D+00    |proj g|=  6.18224D-02

At iterate    1    f=  4.30327D+00    |proj g|=  3.52966D-02

At iterate    2    f=  4.15236D+00    |proj g|=  1.90886D-02

At iterate    3    f=  4.08395D+00    |proj g|=  1.55927D-02

At iterate    4    f=  3.94241D+00    |proj g|=  1.07945D-02

At iterate    5    f=  3.91360D+00    |proj g|=  4.44355D-02

At iterate    6    f=  3.86842D+00    |proj g|=  1.46312D-02

At iterate    7    f=  3.84874D+00    |proj g|=  6.66120D-03

At iterate    8    f=  3.83290D+00    |proj g|=  8.71998D-03

At iterate    9    f=  3.81151D+00    |proj g|=  9.42494D-03

At iterate   10    f=  3.80591D+00    |proj g|=  1.40365D-02

At iterate   11    f=  3.79222D+00    |proj g|=  4.78986D-03

At iterate   12    f=  3.78918D+00    |proj g|=  2.90594D-03

At iterate   13    f=  3

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       462722     M =           10

At X0       967 variables are exactly at the bounds

At iterate    0    f=  3.77847D+00    |proj g|=  1.04221D-02

At iterate    1    f=  3.77715D+00    |proj g|=  4.13257D-03

At iterate    2    f=  3.77613D+00    |proj g|=  1.64209D-03

At iterate    3    f=  3.77605D+00    |proj g|=  2.35587D-03

At iterate    4    f=  3.77587D+00    |proj g|=  9.58074D-04

At iterate    5    f=  3.77583D+00    |proj g|=  4.14812D-04

At iterate    6    f=  3.77578D+00    |proj g|=  9.33336D-04

At iterate    7    f=  3.77574D+00    |proj g|=  4.84205D-04

At iterate    8    f=  3.77571D+00    |proj g|=  1.05320D-03

At iterate    9    f=  3.77570D+00    |proj g|=  5.02707D-04

At iterate   10    f=  3.77569D+00    |proj g|=  2.42768D-04

At iterate   11    f=  3.77568D+00    |proj g|=  3.01617D-04

At iterate   12    f=  3.77567D+00    |proj g|=  4.02543D-04

At iterate   13    f=  3.7


At iterate   51    f=  3.78725D+00    |proj g|=  1.35131D-04

At iterate   52    f=  3.78725D+00    |proj g|=  1.96498D-04

At iterate   53    f=  3.78725D+00    |proj g|=  9.45215D-05

At iterate   54    f=  3.78725D+00    |proj g|=  3.13111D-04

At iterate   55    f=  3.78725D+00    |proj g|=  1.24176D-04

At iterate   56    f=  3.78725D+00    |proj g|=  1.13489D-04

At iterate   57    f=  3.78725D+00    |proj g|=  1.81669D-04

At iterate   58    f=  3.78725D+00    |proj g|=  3.12886D-04

At iterate   59    f=  3.78725D+00    |proj g|=  8.33043D-05

At iterate   60    f=  3.78725D+00    |proj g|=  4.03242D-05

At iterate   61    f=  3.78725D+00    |proj g|=  9.38474D-05

At iterate   62    f=  3.78725D+00    |proj g|=  8.92098D-05

At iterate   63    f=  3.78725D+00    |proj g|=  1.44907D-04

At iterate   64    f=  3.78725D+00    |proj g|=  4.74802D-05

At iterate   65    f=  3.78725D+00    |proj g|=  2.44258D-05

At iterate   66    f=  3.78725D+00    |proj g|=  1.34998D-04

At iter


At iterate   82    f=  3.80382D+00    |proj g|=  1.90066D-04

At iterate   83    f=  3.80382D+00    |proj g|=  3.29923D-04

At iterate   84    f=  3.80382D+00    |proj g|=  5.83169D-04

At iterate   85    f=  3.80382D+00    |proj g|=  1.01002D-03

At iterate   86    f=  3.80382D+00    |proj g|=  1.06656D-04

At iterate   87    f=  3.80382D+00    |proj g|=  3.02774D-04

At iterate   88    f=  3.80382D+00    |proj g|=  4.74089D-04

At iterate   89    f=  3.80382D+00    |proj g|=  9.91459D-04

At iterate   90    f=  3.80382D+00    |proj g|=  2.80668D-04

At iterate   91    f=  3.80381D+00    |proj g|=  8.82325D-05

At iterate   92    f=  3.80381D+00    |proj g|=  2.65754D-04

At iterate   93    f=  3.80381D+00    |proj g|=  3.49308D-04

At iterate   94    f=  3.80381D+00    |proj g|=  4.74474D-04

At iterate   95    f=  3.80381D+00    |proj g|=  1.50483D-04

At iterate   96    f=  3.80381D+00    |proj g|=  6.62885D-05

At iterate   97    f=  3.80381D+00    |proj g|=  1.76743D-04

At iter


At iterate   32    f=  3.81541D+00    |proj g|=  8.05601D-04

At iterate   33    f=  3.81541D+00    |proj g|=  2.48922D-03

At iterate   34    f=  3.81540D+00    |proj g|=  6.33955D-04

At iterate   35    f=  3.81540D+00    |proj g|=  2.82909D-04

At iterate   36    f=  3.81540D+00    |proj g|=  4.90920D-04

At iterate   37    f=  3.81540D+00    |proj g|=  8.01458D-04

At iterate   38    f=  3.81540D+00    |proj g|=  1.31714D-03

At iterate   39    f=  3.81540D+00    |proj g|=  4.73539D-04

At iterate   40    f=  3.81540D+00    |proj g|=  2.29744D-04

At iterate   41    f=  3.81539D+00    |proj g|=  7.42835D-04

At iterate   42    f=  3.81539D+00    |proj g|=  1.14225D-03

At iterate   43    f=  3.81539D+00    |proj g|=  3.19875D-04

At iterate   44    f=  3.81539D+00    |proj g|=  2.47968D-04

At iterate   45    f=  3.81539D+00    |proj g|=  4.76880D-04

At iterate   46    f=  3.81539D+00    |proj g|=  5.77065D-04

At iterate   47    f=  3.81539D+00    |proj g|=  8.08681D-04

At iter

...


At iterate  859    f=  3.86322D+00    |proj g|=  1.31765D-03

At iterate  860    f=  3.86322D+00    |proj g|=  8.04937D-03

At iterate  861    f=  3.86322D+00    |proj g|=  1.08595D-03

At iterate  862    f=  3.86322D+00    |proj g|=  1.64803D-03

At iterate  863    f=  3.86322D+00    |proj g|=  2.24559D-03

At iterate  864    f=  3.86322D+00    |proj g|=  2.67734D-03

At iterate  865    f=  3.86322D+00    |proj g|=  3.84185D-03

At iterate  866    f=  3.86322D+00    |proj g|=  1.48094D-03

At iterate  867    f=  3.86322D+00    |proj g|=  1.87250D-03

At iterate  868    f=  3.86322D+00    |proj g|=  3.54568D-03

At iterate  869    f=  3.86322D+00    |proj g|=  1.66635D-03

At iterate  870    f=  3.86322D+00    |proj g|=  1.47032D-03

At iterate  871    f=  3.86322D+00    |proj g|=  2.74219D-03

At iterate  872    f=  3.86322D+00    |proj g|=  4.17877D-03

At iterate  873    f=  3.86322D+00    |proj g|=  1.41292D-03

At iterate  874    f=  3.86322D+00    |proj g|=  1.51542D-03

At iter


At iterate  992    f=  3.86321D+00    |proj g|=  6.30862D-04

At iterate  993    f=  3.86321D+00    |proj g|=  2.41759D-03

At iterate  994    f=  3.86321D+00    |proj g|=  3.29631D-03

At iterate  995    f=  3.86321D+00    |proj g|=  9.62919D-04

At iterate  996    f=  3.86321D+00    |proj g|=  8.54925D-04

At iterate  997    f=  3.86321D+00    |proj g|=  1.64746D-03

At iterate  998    f=  3.86321D+00    |proj g|=  2.57349D-03

At iterate  999    f=  3.86321D+00    |proj g|=  9.26776D-04

At iterate 1000    f=  3.86321D+00    |proj g|=  9.71843D-04

At iterate 1001    f=  3.86321D+00    |proj g|=  1.67593D-03

At iterate 1002    f=  3.86321D+00    |proj g|=  4.65454D-03

At iterate 1003    f=  3.86321D+00    |proj g|=  7.29663D-04

At iterate 1004    f=  3.86321D+00    |proj g|=  7.85062D-04

At iterate 1005    f=  3.86321D+00    |proj g|=  1.31955D-03

At iterate 1006    f=  3.86321D+00    |proj g|=  3.73814D-03

At iterate 1007    f=  3.86321D+00    |proj g|=  2.05730D-03

At iter


At iterate 1125    f=  3.86320D+00    |proj g|=  1.16388D-03

At iterate 1126    f=  3.86320D+00    |proj g|=  1.63493D-03

At iterate 1127    f=  3.86320D+00    |proj g|=  2.02083D-03

At iterate 1128    f=  3.86320D+00    |proj g|=  4.64054D-03

At iterate 1129    f=  3.86320D+00    |proj g|=  8.98185D-04

At iterate 1130    f=  3.86320D+00    |proj g|=  6.80022D-04

At iterate 1131    f=  3.86320D+00    |proj g|=  1.40586D-03

At iterate 1132    f=  3.86320D+00    |proj g|=  2.18221D-03

At iterate 1133    f=  3.86320D+00    |proj g|=  2.17935D-03

At iterate 1134    f=  3.86320D+00    |proj g|=  6.37791D-04

At iterate 1135    f=  3.86320D+00    |proj g|=  1.14713D-03

At iterate 1136    f=  3.86320D+00    |proj g|=  2.25565D-03

At iterate 1137    f=  3.86320D+00    |proj g|=  1.09350D-03

At iterate 1138    f=  3.86320D+00    |proj g|=  1.00313D-03

At iterate 1139    f=  3.86320D+00    |proj g|=  1.29674D-03

At iterate 1140    f=  3.86320D+00    |proj g|=  3.70092D-03

At iter


At iterate 1258    f=  3.86320D+00    |proj g|=  7.14564D-04

At iterate 1259    f=  3.86320D+00    |proj g|=  8.91191D-04

At iterate 1260    f=  3.86320D+00    |proj g|=  1.25553D-03

At iterate 1261    f=  3.86320D+00    |proj g|=  1.68036D-03

At iterate 1262    f=  3.86320D+00    |proj g|=  5.96414D-04

At iterate 1263    f=  3.86320D+00    |proj g|=  9.96843D-04

At iterate 1264    f=  3.86320D+00    |proj g|=  9.91556D-04

At iterate 1265    f=  3.86320D+00    |proj g|=  2.71710D-03

At iterate 1266    f=  3.86320D+00    |proj g|=  6.25243D-04

At iterate 1267    f=  3.86320D+00    |proj g|=  8.72786D-04

At iterate 1268    f=  3.86320D+00    |proj g|=  9.73845D-04

At iterate 1269    f=  3.86320D+00    |proj g|=  2.18507D-03

At iterate 1270    f=  3.86320D+00    |proj g|=  2.90165D-03

At iterate 1271    f=  3.86320D+00    |proj g|=  4.54651D-04

At iterate 1272    f=  3.86320D+00    |proj g|=  3.56845D-04

At iterate 1273    f=  3.86320D+00    |proj g|=  1.63195D-03

At iter


At iterate 1391    f=  3.86319D+00    |proj g|=  3.51936D-04

At iterate 1392    f=  3.86319D+00    |proj g|=  4.31337D-04

At iterate 1393    f=  3.86319D+00    |proj g|=  1.33730D-03

At iterate 1394    f=  3.86319D+00    |proj g|=  9.03004D-04

At iterate 1395    f=  3.86319D+00    |proj g|=  1.20126D-03

At iterate 1396    f=  3.86319D+00    |proj g|=  3.94316D-04

At iterate 1397    f=  3.86319D+00    |proj g|=  4.48204D-04

At iterate 1398    f=  3.86319D+00    |proj g|=  7.31642D-04

At iterate 1399    f=  3.86319D+00    |proj g|=  3.10721D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****   1399   1446   